#  [DCGAN: Deep Convolutional Generative Adversarial Networks](https://arxiv.org/abs/1511.06434)

**DCGAN** is one of the most popular and succesful network design for GAN. It mainly composes of convolution layers without max pooling or fully connected layers. It uses strided convolutions and transposed convolutions for the downsampling and the upsampling respectively.

**Network Design of DCGAN:**

* Replace all pooling layers with strided convolutions.
* Remove all fully connected layers.
* Use transposed convolutions for upsampling.
* Use Batch Normalization after every layer except after the output layer of the generator and the input layer of the discriminator.
* Use ReLU non-linearity for each layer in the generator except for output layer use tanh.
* Use Leaky-ReLU non-linearity for each layer of the disciminator excpet for output layer use sigmoid.


![](./Sources/dcgan.png)

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [27]:
params={'batch_size':32, 'latent':100, 'nz':100, 'nc':3 ,'ngf':128, 'ndf':128}

# Define the Generator Network

In [28]:
class Generator(nn.Module):
    def __init__(self, params):
        super().__init__()

        # Input is the latent vector Z.
        self.tconv1 = nn.ConvTranspose2d(params['nz'], params['ngf']*8, kernel_size=4, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(params['ngf']*8)

        # Input Dimension: (ngf*8) x 4 x 4
        self.tconv2 = nn.ConvTranspose2d(params['ngf']*8, params['ngf']*4, kernel_size=4, stride=2, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(params['ngf']*4)

        # Input Dimension: (ngf*4) x 8 x 8
        self.tconv3 = nn.ConvTranspose2d(params['ngf']*4, params['ngf']*2, kernel_size=4, stride=2, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(params['ngf']*2)

        # Input Dimension: (ngf*2) x 16 x 16
        self.tconv4 = nn.ConvTranspose2d(params['ngf']*2, params['ngf'], kernel_size=4, stride=2, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(params['ngf'])

        # Input Dimension: (ngf) * 32 * 32
        self.tconv5 = nn.ConvTranspose2d(params['ngf'], params['nc'], kernel_size=4, stride=2, padding=1, bias=False)

        #Output Dimension: (nc) x 64 x 64

    def forward(self, x):
        x = F.relu(self.bn1(self.tconv1(x)))
        x = F.relu(self.bn2(self.tconv2(x)))
        x = F.relu(self.bn3(self.tconv3(x)))
        x = F.relu(self.bn4(self.tconv4(x)))

        x = torch.tanh(self.tconv5(x))

        return x

In [29]:
generator = Generator(params)

In [30]:
dummy_data =torch.rand(params['batch_size'],params['latent'],1,1)
output = generator(dummy_data)
print(f'Input shape: {dummy_data.shape}, Output shape: {output.shape}')

Input shape: torch.Size([32, 100, 1, 1]), Output shape: torch.Size([32, 3, 64, 64])


# Define the Discriminator Network

In [34]:
class Discriminator(nn.Module):
    def __init__(self, params):
        super().__init__()

        # Input Dimension: (nc) x 64 x 64
        self.conv1 = nn.Conv2d(params['nc'], params['ndf'], kernel_size=4, stride=2, padding=1, bias=False)

        # Input Dimension: (ndf) x 32 x 32
        self.conv2 = nn.Conv2d(params['ndf'], params['ndf']*2,  kernel_size=4, stride=2, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(params['ndf']*2)

        # Input Dimension: (ndf*2) x 16 x 16
        self.conv3 = nn.Conv2d(params['ndf']*2, params['ndf']*4, kernel_size=4, stride=2, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(params['ndf']*4)

        # Input Dimension: (ndf*4) x 8 x 8
        self.conv4 = nn.Conv2d(params['ndf']*4, params['ndf']*8, kernel_size=4, stride=2, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(params['ndf']*8)

        # Input Dimension: (ndf*8) x 4 x 4
        self.conv5 = nn.Conv2d(params['ndf']*8, 1, kernel_size=4, stride=1, padding=0, bias=False)

    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), 0.2, True)
        x = F.leaky_relu(self.bn2(self.conv2(x)), 0.2, True)
        x = F.leaky_relu(self.bn3(self.conv3(x)), 0.2, True)
        x = F.leaky_relu(self.bn4(self.conv4(x)), 0.2, True)

        x = torch.sigmoid(self.conv5(x))

        return x

In [35]:
discriminator = Discriminator(params)

In [36]:
dummy_data =torch.rand(params['batch_size'],params['nc'],64,64)
output = discriminator(dummy_data)
print(f'Input shape: {dummy_data.shape}, Output shape: {output.shape}')

Input shape: torch.Size([32, 3, 64, 64]), Output shape: torch.Size([32, 1, 1, 1])


# Do Configuration

In [ ]:
# Apply the weights_init() function 

# Define loss function of the discriminator e.g. criterion = nn.BCELoss()
# Define loss function of the generator e.g. criterion = nn.BCELoss()

# Determine optimizer for the discriminator.
# Determine optimizer for the generator.

# Do training

In [ ]:
# for epoch in range(n_epochs):
      # for i, data in enumerate(dataloader, 0):
              # Train discriminator
              # Train generator
              # Check progress of training.  
              # Save the losses for plotting.
              # Check how the generator is doing by saving G's output on a fixed noise.     
              # Save the model.

# Save the final trained model.